# 数据库简介

**数据库**（**database**）是目前最为成熟的数据存储、查询工具。历史上，发展最为悠久的数据库是所谓的关系型数据库，这类数据库通常依赖于**结构化查询语言**（**SQL**），典型的如MySQL、Oracle、SQLServer以及Postgresql等。

而目前，随着大数据的发展，非关系型的数据库（NoSQL）也获得了蓬勃的发展，这些数据库很多都是以类似字典的键-值类型，包括MongoDB等等。

在这里，我们以Python自带的SQLite为例，简单介绍SQL的用法。

## Python中的SQLite

在Python中，为了方便，自带了一个关系型数据库：SQLite。为了使用这个数据库，首先要将相关模块导入：
```python
import sqlite3
```

之后需要链接数据库。在SQLite3中，一个数据库对应于一个文件，通常以“.db”的后缀名结尾。比如，如果我们要连接一个文件名为"test.db"的数据库，可以使用：
```python
import sqlite3
conn=sqlite3.connect('test.db')
```

就可以了，该命令会返回一个数据库的连接对象：conn。值得注意的是，与文件一样，有打开必须有关闭，在数据库使用结束后，必须使用：
```python
conn.close()
```

关闭数据库。

此外，Python还支持打开内存数据库，内存数据库即数据库存放在内存而非硬盘上，通常用于临时数据的存储，一旦程序结束，该数据库就会消失。内存数据库使用如下方式打开：
```python
import sqlite3
conn=sqlite3.connect(':memory:')
```

通常打开数据库后，conn仅仅是一个连接，并不能执行数据库操作，我们还需要**游标**（**cursor**），对数据库的操作都是使用游标完成的。为了获得游标，可以使用conn的cursor()方法：
```python
import sqlite3
conn=sqlite3.connect('test.db')
cur=conn.cursor()
## 一些操作
cur.close()
conn.close()
```

同样，游标用完要记得关掉。

为了执行SQL语句，一般可以使用游标的execute()方法：
```python
import sqlite3
conn=sqlite3.connect('test.db')
cur=conn.cursor()
cur.execute("SQL语句")
#如果涉及到修改数据库，需要执行： conn.commit()
cur.close()
conn.close()
```

就可以执行SQL语句了。值得注意的是，在cur.execute()执行结束之后，如果涉及到对数据库的任何修改，必须使用conn.commit()提交任务，否则修改不会生效。

## 表、创建表

数据库是由表（table）构成的，一个数据库可以包含多个表，关系型数据库意味着不同表之间存在着紧密的联系。限于篇幅，我们在这里将不再深究表和表之间的关系，仅介绍最简单的数据库操作。

对于表的操作第一个是创建表。表是一个二维结构，每一行代表一个数据，而每一列代表一个变量。在数据库的术语中，列通常称为字段。字段有其数据类型，不同数据库的数据类型差别很大，不过基本的数据类型包括：

* INTEGER：整数
* REAL：实数
* CHAR(n)：长度为n的字符串
* VARCHAR(n)：长度最多为n的字符串
* TEXT：长字符串
* DATE：日期
* DATETIME：日期、时间


创建表格的基本SQL命令为：

```sql
CREATE TABLE IF NOT EXISTS table_name(
    id INTEGER PRIMARY KEY,
    col1 INTEGER NOT NULL,
    col2 VARCHAR(10),
    col3 DATE,
    col4 TEXT,
    ......
);
```

其中，“CREATE TABLE ”代表要创建一个表格，“IF NOT EXISTS”为如果表格不存在则创建，“table_name”为表名。接着，括号里面描述了每一个字段的数据类型和特征，其中：

* PRIMARY KEY 代表主键，在一张表里面一般必须有主键，且主键是唯一识别的
* NOT NULL 代表该字段不能为空
* id, col1,...为字段名

比如，我们可以使用如下简单的SQL语句创建一个表：

In [1]:
import sqlite3
conn=sqlite3.connect('test.db')
cur=conn.cursor()
createTable=("CREATE TABLE IF NOT EXISTS player ("
             "id INTEGER PRIMARY KEY AUTOINCREMENT, "
             "name VARCHAR(500) NOT NULL,"
             "code INTEGER);")
print(createTable)
conn.execute(createTable)
conn.commit()
cur.close()
conn.close()

CREATE TABLE IF NOT EXISTS player (id INTEGER PRIMARY KEY AUTOINCREMENT, name VARCHAR(500) NOT NULL,code INTEGER);


## 插入数据

插入数据的基本SQL命令为：
```sql
INSERT INTO table_name(col1,col3,...) VALUES(?,?,...);
```

比如，如下命令可以向刚刚的表添加一行数据：
```sql
INSERT INTO table_name(name,code) VALUES("Messi",10);
```

在Python中，cur.execute()允许如下的语法：
```python
cur.execute("INSERT INTO table_name(name,code) VALUES(?,?)",("Messi",10))
```

其中每个问号代表了后面元组的一个数据。比如：

In [2]:
player_list=[('Messi', 10), ('Xavi', 6), ('ter Stegen', 1), ('Busquets', 5), ('Pique', 3), ('Suárez', 9)]
conn=sqlite3.connect('test.db')
cur=conn.cursor()
colname=['name','code']
sql="INSERT INTO player("+','.join(colname)+") VALUES("+','.join('?'*len(colname))+")"
print(sql)
for p in player_list:
    cur.execute(sql,p)
conn.commit()
cur.close()
conn.close()

INSERT INTO player(name,code) VALUES(?,?)


如上，我们为这个表中插入了6条数据。

## 查询数据

查询是SQL中最常用的命令。一般的语法为：

```sql
SELECT col1,col2,... FROM table_name WHERE conditions
```

比如，如果我们想要查找id=1的所有字段的数据，可以使用：

```sql
SELECT * FROM player WHERE id=1
```

或者，如果我们需要找Messi的号码，可以使用：
```sql
SELECT code FROM player WHERE name='Messi'
```

或者，如果我们需要找所有小于10的号码的id和姓名，可以使用：
```sql
SELECT id,name FROM player WHERE code<10
```

使用cursor.execute()执行以上的SQL命令之后，会返回一个可以迭代的对象，每次迭代返回一个元组，该元组为查询的结果。比如：

In [3]:
conn=sqlite3.connect('test.db')
cur=conn.cursor()
colname=['name','code']
sql="SELECT id,name FROM player WHERE code<10"
result=cur.execute(sql)
for r in result:
    print("id=%d, name=%s" % r)
cur.close()
conn.close()

id=2, name=Xavi
id=3, name=ter Stegen
id=4, name=Busquets
id=5, name=Pique
id=6, name=Suárez


或者，也可以使用fetchall()命令，返回所有结果的列表：

In [4]:
conn=sqlite3.connect('test.db')
cur=conn.cursor()
colname=['name','code']
sql="SELECT id,name FROM player WHERE code<10"
result=cur.execute(sql)
print(result.fetchall())
cur.close()
conn.close()

[(2, 'Xavi'), (3, 'ter Stegen'), (4, 'Busquets'), (5, 'Pique'), (6, 'Suárez')]
